In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm

In [3]:
# Hyper parameters
EPISODES = 50 # times every setting is replicated
IF_WRITEFILE = True
#IF_WRITEFILE = False
RESULT_ADD = 'M3distributedSimulation.csv'

In [4]:
# initialize parameters
R = {
        'lambda0' : 10.0, # lambdas are customer arriving rates at each block
        'lambda1' : 10.0,
        'theta' : 10,
        'phi' : 5.0,      # rate at fixing queue at each area
        'f_server' : 1,
        'broken' : 0.3,     # probability of a bike to be broken after each ride
        'step' : 0.5, # carrier moving rate among areas
       }

#queues: A0, A1, C, F, D, R00, R01, R10, R11
P = [[0.3, 0.7],
       [0.7, 0.3]]

In [9]:
# theoretical distributed QN
# simulate situation with waiting and fixing
# in this version, bikes found broken after a customer RIDING


# BikeNet is the class for simulate the CQN
# simulate by discrete event, in this program called a step
# the average customer served on time is out research object


class BikeNet():
    def __init__(self, N, A, R, P, warmup_time, run_time):
        '''
        N: number of bikes in the CQN
        A: number of areas in the CQN
        R: rates of the system
        P: matrix of transit probability, including to broken
        warmup_time: the time to warm up the system
        run_time: the period of time during which get the performance
        '''
        
        self.N = N
        self.A = A
        self.R = R
        self.P = P
        self.warmup_time = warmup_time
        self.run_time = run_time
        self.time_limit = warmup_time + run_time

        self.areas = list(range(A))

        self.served_customers = 0

    def reset(self):
        self.T = 0 # time cursor to indicate the system time in this episode, at the end of every episode should be default to zero
        self.formerT = 0
        self.served_customers = 0 # the performance indicator
        self.F = []
        heapq.heapify(self.F) #time to be available for fixing queue at each area
        # states: devide into 2 parts
        # A0, ..., AA, W0, ..., WA, F0, ..., FA: 1-d list
        # [[R00, ..., R(0,A-1)1], ..., [R(A-1,0), ..., R(A-1,A-1)]]: 2-d list for all riding queues
        #self.state = [int(self.N/self.A)]*self.A + [0]*(self.A**2+2*self.A)
        self.state1 = [int(self.N/self.A)]*self.A + [0]*self.A # 3 queues for each area
        self.I = [1]+[0]*(self.A-1)
        self.state2 = [[0]*self.A for i in range(self.A)]
        self.stateRecord = self.state1 + self.I + self.state2[0] + self.state2[1]
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(self.A):
            # state: [time, type, start, terminal]
            # initially generate events of arriving at each area
            heapq.heappush(self.scheduler, [random.expovariate(self.R['lambda'+str(i)]), -1, i, 0])
        heapq.heappush(self.scheduler, [0, 2, 0, 0])
        #return self.state + [self.T]
    
    def getRecord(self):
        result = []
        result = self.stateRecord
        result.append((self.T - self.formerT)/self.time_limit)
        self.formerT = self.T
        self.stateRecord = self.state1 + self.I + self.state2[0] + self.state2[1]
        #print(result)
        return result
    
    def simulate(self, writefile):
        if writefile:
            with open(RESULT_ADD, 'a') as fout:
                writer = csv.writer(fout)
                self.reset()
                while self.T <= self.time_limit:
                    self.step()
                    writer.writerow(self.getRecord())
        else:
            self.reset()
            while self.T <= self.time_limit:
                self.step()
                #if sum(self.state1)+sum([sum(_) for _ in self.state2]) != self.N:
                #print(self.state1, self.state2)
            return self.served_customers / (self.T-self.warmup_time)
                
    def get_riding_time(self, s, t):
        rowS, colS, rowT, colT = s//self.A, s%self.A, t//self.A, t%self.A
        if s==t: r = 0.5
        else: r = abs(rowS-rowT) + abs(colS-colT)
        return random.expovariate(r)
                
        
    def add_event(self, kind, t):
        if kind == 2: 
            #print(t, len(self.W[t]))
            next_time = random.expovariate(self.R['theta']) if self.R['theta']!=0 else 0
            next_time += self.T
        elif kind == 3: 
            next_time = random.expovariate(self.R['phi'])
            next_time += self.T 

        heapq.heappush(self.scheduler, [next_time, kind, t, t])
        
    
    def step(self):

        event = self.scheduler[0]
        self.T, kind, start, terminal = event[0], event[1], event[2], event[3]

        '''kind:
        -1: customer ride a bike away
         1: a bike arrives
         2: a carrier arrives
         3: a bike was fixed
        '''
        if kind == 1: # 顾客骑行到达
            self.state2[start][terminal] -= 1
            if random.random()<self.R['broken']:
                if sum(self.state1[self.A:]) == 0:
                    self.add_event(2,0)
                self.state1[terminal+self.A] += 1
                heapq.heappop(self.scheduler)
            else:
                self.state1[terminal] += 1
                heapq.heappop(self.scheduler)
        elif kind == 2:
            #print('ever 2')
            heapq.heappop(self.scheduler)
            temp = [0]*self.A
            temp[start] = 1
            self.I = temp
            if self.state1[self.A+start]>0:
                self.add_event(3, start)
            elif sum(self.state1[self.A:]) != 0:
                self.add_event(2, (start+1)%self.A)
        elif kind == 3:
            #print('ever 3')
            self.state1[self.A+start] -= 1
            self.state1[terminal] += 1
            heapq.heappop(self.scheduler)
            if self.state1[self.A+start] == 0:
                if sum(self.state1[self.A:])!=0:
                    self.add_event(2, (start+1)%self.A)
            else:
                self.add_event(3, start)
        else:# 顾客到达
            if self.state1[start] == 0:  # 但没车
                heapq.heappop(self.scheduler)
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])
            else:
                if self.T>self.warmup_time: 
                    self.served_customers += 1
                    #result.append(self.served_customers / (self.T-self.warmup_time))
                # if self.T>self.warmup_time: result.append(self.T)
                
                self.state1[start] -= 1
                target = random.choices(self.areas, weights=self.P[start], k=1)[0]
                self.state2[start][target] += 1
                
                heapq.heappop(self.scheduler)
                next_time = self.get_riding_time(start, target) + self.T
                heapq.heappush(self.scheduler, [next_time, 1, start, target])
                next_time = random.expovariate(self.R['lambda'+str(start)]) + self.T
                heapq.heappush(self.scheduler, [next_time, -1, start, 0])


        #return self.state+[self.T]

random.seed(1)
N = 3  # total number of bikes in the QN
A = 2  # A for areas, indicates the number of areas and the action space


WARMUP_TIME = 200
RUN_TIME = 500

env = BikeNet(N=N,
              A=A,
              R=R,
              P=P,
              warmup_time=WARMUP_TIME,
              run_time=RUN_TIME)

%time env.simulate(IF_WRITEFILE)

# result = []
# for i in tqdm(range(1,41)):
#     env.R['carmove'] = i*0.5
#     result.append(env.simulate())
    
# plt.plot(result)
# plt.show()

CPU times: user 108 ms, sys: 3.93 ms, total: 112 ms
Wall time: 116 ms
